In [1]:
import pandas as pd
import gc
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score, classification_report, f1_score
import random 
import warnings
import transformers
import torch

warnings.filterwarnings('ignore', category=FutureWarning)
transformers.logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

Using device: mps


### Load data

In [2]:
# Load as HuggingFace dataset
dataset = Dataset.from_csv('../data/cleaned_for_LLM.csv')
print(f"Loaded {len(dataset):,} rows")

# Filter out manual labeling set
absa_ids = set(pd.read_csv('../data/absa_training_set.csv', usecols=['review_id'])['review_id'].astype(str))
dataset = dataset.filter(lambda x: x['review_id'] not in absa_ids)
print(f"After filtering absa set: {len(dataset):,} rows")

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Loaded 22,624,379 rows


Filter:   0%|          | 0/22624379 [00:00<?, ? examples/s]

After filtering absa set: 22,614,379 rows


### Evaluation Metrics

In [3]:
# Define metrics
def compute_metrics(pred):
    """Compute evaluation metrics for multi-label classification."""
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)
    
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

## RoBERTa Tokenization

In [4]:
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

aspect_cols = [
             'product_quality_positive', 'product_quality_negative',
             'service_positive', 'service_negative',
             'wait_time_positive', 'wait_time_negative',
             'price_value_positive', 'price_value_negative',
             'cleanliness_positive', 'cleanliness_negative',
             'atmosphere_positive', 'atmosphere_negative',
             'general_positive', 'general_negative'
             ]

def tokenize_function(examples):
    """Tokenize text and prepare multi-label targets."""
    tokenized = tokenizer(
        examples['text'],
        padding='longest',  # only pads longest batch, not always full 128
        truncation=True,
        max_length=128     
    )
    labels = []
    for i in range(len(examples['text'])):
        label_row = [float(examples[col][i]) for col in aspect_cols]
        labels.append(label_row)
    tokenized['labels'] = labels
    return tokenized

tokenized = dataset.map(tokenize_function, batched=True, remove_columns=dataset.column_names)
tokenized.save_to_disk('../data/roberta_tokenized')

Map:   0%|          | 0/22614379 [00:00<?, ? examples/s]

Saving the dataset (0/35 shards):   0%|          | 0/22614379 [00:00<?, ? examples/s]

***Note:*** 0% is a display bug in the notebook/library version.

## RoBERTa-base (25k Sample)

### Train

In [10]:
# Configuration for each run
SAMPLE_SIZE   = 25_000
TEST_SIZE     = 0.1
EPOCHS        = 3
TRAIN_BATCH   = 64
EVAL_BATCH    = 128
LEARNING_RATE = 2e-5
SEED          = 2
BALANCE_CLASSES = None
CLASS_WEIGHTS = None

# Sample & Split
random.seed(SEED)
sample       = tokenized_dataset.select(random.sample(range(len(tokenized_dataset)), SAMPLE_SIZE))
split        = sample.train_test_split(test_size=TEST_SIZE, seed=SEED)
train_subset = split['train']
test_subset  = split['test']

print(f"Train subset: {len(train_subset):,}")
print(f"Test subset:  {len(test_subset):,}")

# Load RoBERTa-base Model 
print("Loading RoBERTa model...")
model = RobertaForSequenceClassification.from_pretrained(
    "roberta-base",
    num_labels=len(aspect_cols),
    problem_type="multi_label_classification"
)
print(f"Model loaded with {len(aspect_cols)} output labels\n")

# Training Arguments
training_args = TrainingArguments(
    output_dir=f'./results_roberta_{SAMPLE_SIZE}',
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='f1',
    logging_steps=100,
    per_device_train_batch_size=TRAIN_BATCH,
    per_device_eval_batch_size=EVAL_BATCH,
    num_train_epochs=EPOCHS,
    learning_rate=LEARNING_RATE,
    fp16=torch.cuda.is_available(),
    dataloader_pin_memory=torch.cuda.is_available(),
    dataloader_num_workers=4,
)

# Train Model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=test_subset,
    compute_metrics=compute_metrics,
)
print("Starting training...\n")
trainer.train()

Loading RoBERTa model...


Loading weights:   0%|          | 0/197 [00:00<?, ?it/s]

RobertaForSequenceClassification LOAD REPORT from: roberta-base
Key                             | Status     | 
--------------------------------+------------+-
roberta.embeddings.position_ids | UNEXPECTED | 
lm_head.layer_norm.bias         | UNEXPECTED | 
lm_head.bias                    | UNEXPECTED | 
lm_head.dense.weight            | UNEXPECTED | 
lm_head.layer_norm.weight       | UNEXPECTED | 
lm_head.dense.bias              | UNEXPECTED | 
classifier.dense.bias           | MISSING    | 
classifier.out_proj.bias        | MISSING    | 
classifier.dense.weight         | MISSING    | 
classifier.out_proj.weight      | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


Model loaded with 18 output labels

Starting training...



Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.060443,0.057929,0.835200,0.782257,0.814565,0.783551
2,0.046927,0.045346,0.857200,0.836677,0.887658,0.821060
3,0.037241,0.045397,0.867200,0.844941,0.877104,0.832072


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

There were missing keys in the checkpoint model loaded: ['roberta.embeddings.LayerNorm.weight', 'roberta.embeddings.LayerNorm.bias', 'roberta.encoder.layer.0.attention.output.LayerNorm.weight', 'roberta.encoder.layer.0.attention.output.LayerNorm.bias', 'roberta.encoder.layer.0.output.LayerNorm.weight', 'roberta.encoder.layer.0.output.LayerNorm.bias', 'roberta.encoder.layer.1.attention.output.LayerNorm.weight', 'roberta.encoder.layer.1.attention.output.LayerNorm.bias', 'roberta.encoder.layer.1.output.LayerNorm.weight', 'roberta.encoder.layer.1.output.LayerNorm.bias', 'roberta.encoder.layer.2.attention.output.LayerNorm.weight', 'roberta.encoder.layer.2.attention.output.LayerNorm.bias', 'roberta.encoder.layer.2.output.LayerNorm.weight', 'roberta.encoder.layer.2.output.LayerNorm.bias', 'roberta.encoder.layer.3.attention.output.LayerNorm.weight', 'roberta.encoder.layer.3.attention.output.LayerNorm.bias', 'roberta.encoder.layer.3.output.LayerNorm.weight', 'roberta.encoder.layer.3.output.Laye

TrainOutput(global_step=2112, training_loss=0.06492481406100771, metrics={'train_runtime': 6456.9252, 'train_samples_per_second': 10.454, 'train_steps_per_second': 0.327, 'total_flos': 1.776254759424e+16, 'train_loss': 0.06492481406100771, 'epoch': 3.0})

### Evaluate

In [11]:
predictions = trainer.predict(test_subset)
y_pred = (predictions.predictions > 0.5).astype(int)
y_true = predictions.label_ids

print(f"\nRoBERTa ({SAMPLE_SIZE:,} Sample) — Epochs: {EPOCHS} | LR: {LEARNING_RATE} | Batch: {TRAIN_BATCH}")
print(f"Training Accuracy: {trainer.evaluate(train_subset)['eval_accuracy']:.4f}")
print(f"Test Accuracy:     {trainer.evaluate(test_subset)['eval_accuracy']:.4f}")
print(f"\nF1 Score (macro):    {f1_score(y_true, y_pred, average='macro', zero_division=0):.4f}")
print(f"F1 Score (weighted): {f1_score(y_true, y_pred, average='weighted', zero_division=0):.4f}")
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=aspect_cols, zero_division=0))

RoBERTa (25,000 Sample):



Training Accuracy: 0.9095
Test Accuracy:     0.8672

F1 Score (macro):    0.5889
F1 Score (weighted): 0.8449

Classification Report:
              precision    recall  f1-score   support

           0       0.95      0.95      0.95       723
           1       0.72      0.64      0.68        80
           2       0.46      0.14      0.21        88
           3       0.96      0.95      0.96       607
           4       0.84      0.84      0.84       110
           5       0.53      0.16      0.25        50
           6       0.92      0.90      0.91       327
           7       0.83      0.75      0.79        76
           8       0.50      0.10      0.17        50
           9       0.92      0.95      0.93       260
          10       0.86      0.62      0.72        61
          11       0.20      0.02      0.04        44
          12       0.96      0.94      0.95       132
          13       0.89      0.70      0.78        23
          14       0.00      0.00      0.00        16
  

### Clear memory

In [ ]:
del model
del trainer
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None
print("Memory cleared.")